# Molecular Dynamics with Tkinter 

In [1]:
import tkinter as tk
import random
import math

In [2]:
# This adds in a way to draw a circle in a much more
# intuitive way
def _create_circle(self, x, y, r, **kwargs):
    return self.create_oval(x-r, y-r, x+r, y+r, **kwargs)
tk.Canvas.create_circle = _create_circle

In [ ]:
class molecular_dynamics():
    def __init__(self, num_atoms = 10, width=500, height=500):
        '''
           Function to initialize animation. 
           
           Input parameters:
              num_atoms :  the number of atoms to be drawn (default = 10)
              width     :  the width of the window to be created (default = 500)
              height    :  the height of the window to be created (default = 500)
        '''
        # Boiler plate stuff we need to initialize a window
        self.root = tk.Tk()
        self.root.wm_title("Molecular Dynamics with Tkinter")
        self.canvas = tk.Canvas(self.root, width=width, height=height)
        self.canvas.pack()
        
        # Variables we want everyone to have
        self.width = width
        self.height = height
        self.num_atoms = num_atoms
        self.atom_radius = 20
        self.atoms = []
        self.positions = []
        self.positions = []
        self.velocities = []
        self.max_velocity = 20
        self.num_steps = 10000
        self.dt = 0.1
        
        # Now we add in our defined function calls
        self.initialize_atoms()
        self.initialize_velocities()
        
        # This is how we tell Tkinter to draw and
        # animate everything
        self.canvas.pack()
        self.root.after(0, self.animation)
        self.root.mainloop()
   

        
    def initialize_atoms(self):
        '''
           Function to initialize all of our atoms in a smart way 
           (not on top of each other). 
        '''
        # Loop over each atom
        for i in range(self.num_atoms):
            # Creating a random location for the atom
            # We are using the 0.2-0.8 to make sure we're not on the edge
            x = random.uniform(0.2,0.8) * self.width
            y = random.uniform(0.2,0.8) * self.height
            
            # Creating a random color for the atom
            color = random.randint(0, 68719476736)
            color = "#" + hex(color)[2:].zfill(9)
            
            # Save locations inside the list of atoms
            self.positions.append([x,y])
            
            # Lets make sure this atom isn't on top of another atom
            # Lets get the minimum distance to all other atoms
            d = 1000000.0
            for j in range(0,i):
                d = min(d, self.distance(self.positions[i],self.positions[j]))
            
            # Check if we're too close
            while d < 2.0*self.atom_radius:    
                # Reset d
                d = 1000000.0
        
                # Choose a new random spot and remeasure
                x = random.uniform(0.2,0.8) * self.width 
                y = random.uniform(0.2,0.8) * self.height
                
                # Move the atom
                self.positions[i] = [x,y]
                
                # Recalculate the distance
                for j in range(0,i):
                    d = min(d, self.distance(self.positions[i],self.positions[j]))
                    
            # We now know that the location is safe,
            # so lets draw the atom
            self.atoms.append(self.canvas.create_circle(x,
                                                        y,
                                                        self.atom_radius,
                                                        fill=color,
                                                        outline=""))

    def initialize_velocities(self):
        '''
           Intializes a list with num_atoms number of 
           velocities, chosen uniformly from the range
           of [-20, 20].
        '''
        # Now loop over the number of atoms
        for i in range(self.num_atoms):
            # Stored as [x_vel, y_vel] pair
            self.velocities.append([random.uniform(-1,1)*self.max_velocity, 
                                    random.uniform(-1,1)*self.max_velocity])
    
    
    def distance(self, atom1, atom2):   
        '''
           Inputs are the positions of each atom, 
           which contain (x,y) coordinate pairs.
           
           Distance formula is:
              distance = sqrt((x1-x2)^2 + (y1-y2)^2)
        '''

    
        # Lets get the coordinates of atom1
        x1 = atom1[0]
        y1 = atom1[1]
        
        # Lets get the coordinates of atom2
        x2 = atom2[0]
        y2 = atom2[1]
        
        # Lets calculate the distance
        d = math.sqrt((x1-x2)**2 + (y1-y2)**2)
    
        # And lets return our distance
        return d       
    
    
    # Lets name our function
    def bounce_check(self):
        '''
           Function to check if an inter-atom collision
           will occur at the next time step. If one is 
           going to happen, this function will correct the 
           velocities.
        '''
        # This will help us not double count atoms
        # We need to keep track of this index by hand
        # because we are using a for loop later.
        i = 1

        # We need to check for every atom
        # Using a for loop here isn't required, but it 
        # makes the code easier.
        for atom1 in self.positions:
        
            # This will help us not double count
            # (same argument as above)
            j = i
        
            # Compute the distance to every atom ahead of the current one
            for atom2 in self.positions[i:]:
            
                # We will need the distance, so store it
                d = self.distance(atom1,atom2)
            
                # Check if a bounce should occur
                if d < 2.0*self.atom_radius:
                    #print "BOUNCE FOUND: Atom %d hit Atom %d" % (i-1,j)
                    # We need positions of atoms
                    x1 = atom1[0]
                    y1 = atom1[1]
                    x2 = atom2[0]
                    y2 = atom2[1]
                
                    # Now calculate the new velocities
                    # First, calculate the intermediate quantity (v2-v1).R (dot product)
                    inter = ((self.velocities[j][0]-self.velocities[i-1][0])*(x2-x1) + (self.velocities[j][1]-self.velocities[i-1][1])*(y2-y1))/(d**2)
                    #v1R = v_list[i-1][0]*(x1-x2) + v_list[i-1][1]*(y1-y2) #projection of first atom velocity onto R
                    #v2R = v_list[j][0]*(x1-x2) + v_list[j][1]*(y1-y2) #projection of 2nd atom velocity onto R
                
                    # Update new velocities
                    self.velocities[i-1][0] = self.velocities[i-1][0] + inter * (x2-x1)
                    self.velocities[i-1][1] = self.velocities[i-1][1] + inter * (y2-y1)
                    self.velocities[j][0] = self.velocities[j][0] - inter * (x2-x1)
                    self.velocities[j][1] = self.velocities[j][1] - inter * (y2-y1)

                # Update second counter
                j = j + 1
            
            # Update counter
            i = i + 1
            
            
    def wall_check(self):
        '''
           This function checks if at the next time step
           a collision with any of the walls will occur.
           If one is to occur, this will correct the 
           velocities.
        '''
        # Check each atom
        for i in range(self.num_atoms):     
            
            # Check if moving left or right will put our atom beyond the wall
            if abs(self.positions[i][0] + self.dt * self.velocities[i][0]) >= self.width - self.atom_radius or abs(self.positions[i][0] + self.dt * self.velocities[i][0]) <= self.atom_radius:
                
                # We have moved too far right or left, so flip the x_vel
                self.velocities[i][0] = -self.velocities[i][0]           
    
            # Check if moving up or down will put our atom beyond the wall
            if abs(self.positions[i][1] + self.dt * self.velocities[i][1]) >= self.height - self.atom_radius or abs(self.positions[i][1] + self.dt * self.velocities[i][1]) <= self.atom_radius:
                
                # We have moved too far up or down, so flip the y_vel
                self.velocities[i][1] = -self.velocities[i][1]        

    
    
    def animation(self):
        '''
           The function to animate our atoms.
        '''
        # Variables for loop
        i = 0
        
        # Now lets loop through all
        # our steps
        while i < self.num_steps:
            
            # Check if an atom will collide with a wall
            self.wall_check()
            
            # Check if an atom will collide with another
            # atom
            self.bounce_check()
            
            # Move each atom according to its velocity
            for j in range(self.num_atoms):
                self.canvas.move(self.atoms[j],
                                 self.dt * self.velocities[j][0], 
                                 self.dt * self.velocities[j][1])
                # Update the position
                self.positions[j][0] = self.positions[j][0] + self.dt * self.velocities[j][0]
                self.positions[j][1] = self.positions[j][1] + self.dt * self.velocities[j][1]
            
            # Update the canvas (this actually changes the image)
            self.canvas.update()
            
            # Always update our loop counter
            i += 1
            

In [ ]:
molecular_dynamics(num_atoms=20, width=1000, height=500)